In [14]:
import json
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
from scipy import stats
import tweepy
from tweepy import OAuthHandler
import string

## Gather data

In [15]:
WeRateDogs = pd.read_csv('twitter-archive-enhanced.csv')

In [16]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)

with open(url.split('/')[-1], mode = 'wb') as outfile:
    outfile.write(response.content)

# Read the downloaded file into a dataframe 'images'
TweetImage = pd.read_csv('image-predictions.tsv', sep = '\t', encoding = 'utf-8')
# Quick check to the file content and structure
TweetImage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [18]:
# authentication pieces
consumer_key = 'z3CJe9z45B2UsWbpFT2z6bIFs'
consumer_secret = 'EEuXBRsAd1zpT5phHn8mydm56OymrIZPM51VMNGr9kVPmFHilB'
access_token = '136870154-Nq25HjiIkzxXID0dmARgKTnPGmyfjsYj2WQ1OtwF'
access_secret = 'Od80xMlIMcv6cAcbeVlnVfUvcwZLJ9nwi08zyw1qoD44a'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler=auth, 
                 wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

#Twitter Query using tweet_id information from the df.

tweet_ids = list(WeRateDogs.tweet_id)

In [19]:

tweet_data = {}
for tweet in tweet_ids:
    try:
        tweet_status = api.get_status(tweet,
                                      wait_on_rate_limit=True, 
                                      wait_on_rate_limit_notify=True)
        tweet_data[str(tweet)] = tweet_status._json
        print("Valid for: " + str(tweet))
    except: 
        print("Error for: " + str(tweet))

Valid for: 892420643555336193
Valid for: 892177421306343426
Valid for: 891815181378084864
Valid for: 891689557279858688
Valid for: 891327558926688256
Valid for: 891087950875897856
Valid for: 890971913173991426
Valid for: 890729181411237888
Valid for: 890609185150312448
Valid for: 890240255349198849
Valid for: 890006608113172480
Valid for: 889880896479866881
Valid for: 889665388333682689
Valid for: 889638837579907072
Valid for: 889531135344209921
Valid for: 889278841981685760
Valid for: 888917238123831296
Valid for: 888804989199671297
Valid for: 888554962724278272
Error for: 888202515573088257
Valid for: 888078434458587136
Valid for: 887705289381826560
Valid for: 887517139158093824
Valid for: 887473957103951883
Valid for: 887343217045368832
Valid for: 887101392804085760
Valid for: 886983233522544640
Valid for: 886736880519319552
Valid for: 886680336477933568
Valid for: 886366144734445568
Valid for: 886267009285017600
Valid for: 886258384151887873
Valid for: 886054160059072513
Valid for:

Valid for: 840696689258311684
Valid for: 840632337062862849
Valid for: 840370681858686976
Valid for: 840268004936019968
Valid for: 839990271299457024
Valid for: 839549326359670784
Valid for: 839290600511926273
Valid for: 839239871831150596
Valid for: 838952994649550848
Valid for: 838921590096166913
Valid for: 838916489579200512
Valid for: 838831947270979586
Valid for: 838561493054533637
Valid for: 838476387338051585
Valid for: 838201503651401729
Valid for: 838150277551247360
Valid for: 838085839343206401
Valid for: 838083903487373313
Valid for: 837820167694528512
Valid for: 837482249356513284
Valid for: 837471256429613056
Valid for: 837366284874571778
Valid for: 837110210464448512
Error for: 837012587749474308
Valid for: 836989968035819520
Valid for: 836753516572119041
Valid for: 836677758902222849
Valid for: 836648853927522308
Valid for: 836397794269200385
Valid for: 836380477523124226
Valid for: 836260088725786625
Valid for: 836001077879255040
Valid for: 835685285446955009
Valid for:

Valid for: 805207613751304193
Valid for: 804738756058218496
Valid for: 804475857670639616
Valid for: 804413760345620481
Valid for: 804026241225523202
Valid for: 803773340896923648
Valid for: 803692223237865472
Valid for: 803638050916102144
Valid for: 803380650405482500
Valid for: 803321560782307329
Valid for: 803276597545603072
Valid for: 802952499103731712
Valid for: 802624713319034886
Valid for: 802600418706604034
Valid for: 802572683846291456
Valid for: 802323869084381190
Valid for: 802265048156610565
Error for: 802247111496568832
Valid for: 802239329049477120
Valid for: 802185808107208704
Valid for: 801958328846974976
Valid for: 801854953262350336
Valid for: 801538201127157760
Valid for: 801285448605831168
Valid for: 801167903437357056
Valid for: 801127390143516673
Valid for: 801115127852503040
Valid for: 800859414831898624
Valid for: 800855607700029440
Valid for: 800751577355128832
Valid for: 800513324630806528
Valid for: 800459316964663297
Valid for: 800443802682937345
Valid for:

Valid for: 769940425801170949
Valid for: 769695466921623552
Valid for: 769335591808995329
Valid for: 769212283578875904
Valid for: 768970937022709760
Valid for: 768909767477751808
Valid for: 768855141948723200
Valid for: 768609597686943744
Valid for: 768596291618299904
Valid for: 768554158521745409
Valid for: 768473857036525572
Valid for: 768193404517830656
Valid for: 767884188863397888
Valid for: 767754930266464257
Valid for: 767500508068192258
Valid for: 767191397493538821
Valid for: 767122157629476866
Valid for: 766864461642756096
Valid for: 766793450729734144
Valid for: 766714921925144576
Valid for: 766693177336135680
Valid for: 766423258543644672
Valid for: 766313316352462849
Valid for: 766078092750233600
Valid for: 766069199026450432
Valid for: 766008592277377025
Valid for: 765719909049503744
Valid for: 765669560888528897
Valid for: 765395769549590528
Valid for: 765371061932261376
Valid for: 765222098633691136
Valid for: 764857477905154048
Valid for: 764259802650378240
Valid for:

Valid for: 736365877722001409
Valid for: 736225175608430592
Valid for: 736010884653420544
Valid for: 735991953473572864
Valid for: 735648611367784448
Valid for: 735635087207878657
Valid for: 735274964362878976
Valid for: 735256018284875776
Valid for: 735137028879360001
Valid for: 734912297295085568
Valid for: 734787690684657664
Valid for: 734776360183431168
Valid for: 734559631394082816
Valid for: 733828123016450049
Valid for: 733822306246479872
Valid for: 733482008106668032
Valid for: 733460102733135873
Valid for: 733109485275860992
Valid for: 732732193018155009
Valid for: 732726085725589504
Valid for: 732585889486888962
Valid for: 732375214819057664
Valid for: 732005617171337216
Valid for: 731285275100512256
Valid for: 731156023742988288
Valid for: 730924654643314689
Valid for: 730573383004487680
Valid for: 730427201120833536
Valid for: 730211855403241472
Valid for: 730196704625098752
Valid for: 729854734790754305
Valid for: 729838605770891264
Valid for: 729823566028484608
Valid for:

Valid for: 702321140488925184
Valid for: 702276748847800320
Valid for: 702217446468493312
Valid for: 701981390485725185
Valid for: 701952816642965504
Valid for: 701889187134500865
Valid for: 701805642395348998
Valid for: 701601587219795968
Valid for: 701570477911896070
Valid for: 701545186879471618
Valid for: 701214700881756160
Valid for: 700890391244103680
Valid for: 700864154249383937
Valid for: 700847567345688576
Valid for: 700796979434098688
Valid for: 700747788515020802
Valid for: 700518061187723268
Valid for: 700505138482569216
Valid for: 700462010979500032
Valid for: 700167517596164096
Valid for: 700151421916807169
Valid for: 700143752053182464
Valid for: 700062718104104960
Valid for: 700029284593901568
Valid for: 700002074055016451
Valid for: 699801817392291840
Valid for: 699788877217865730
Valid for: 699779630832685056
Valid for: 699775878809702401
Valid for: 699691744225525762
Valid for: 699446877801091073
Valid for: 699434518667751424
Valid for: 699423671849451520
Valid for:

Valid for: 683852578183077888
Valid for: 683849932751646720
Valid for: 683834909291606017
Valid for: 683828599284170753
Valid for: 683773439333797890
Valid for: 683742671509258241
Valid for: 683515932363329536
Valid for: 683498322573824003
Valid for: 683481228088049664
Valid for: 683462770029932544
Valid for: 683449695444799489
Valid for: 683391852557561860
Valid for: 683357973142474752
Valid for: 683142553609318400
Valid for: 683111407806746624
Valid for: 683098815881154561
Valid for: 683078886620553216
Valid for: 683030066213818368
Valid for: 682962037429899265
Valid for: 682808988178739200
Valid for: 682788441537560576
Valid for: 682750546109968385
Valid for: 682697186228989953
Valid for: 682662431982772225
Valid for: 682638830361513985
Valid for: 682429480204398592
Valid for: 682406705142087680
Valid for: 682393905736888321
Valid for: 682389078323662849
Valid for: 682303737705140231
Valid for: 682259524040966145
Valid for: 682242692827447297
Valid for: 682088079302213632
Valid for:

Valid for: 674291837063053312
Valid for: 674271431610523648
Valid for: 674269164442398721
Valid for: 674265582246694913
Valid for: 674262580978937856
Valid for: 674255168825880576
Valid for: 674082852460433408
Valid for: 674075285688614912
Valid for: 674063288070742018
Valid for: 674053186244734976
Valid for: 674051556661161984
Valid for: 674045139690631169
Valid for: 674042553264685056
Valid for: 674038233588723717
Valid for: 674036086168010753
Valid for: 674024893172875264
Valid for: 674019345211760640
Valid for: 674014384960745472
Valid for: 674008982932058114
Valid for: 673956914389192708
Valid for: 673919437611909120
Valid for: 673906403526995968
Valid for: 673887867907739649
Valid for: 673716320723169284
Valid for: 673715861853720576
Valid for: 673711475735838725
Valid for: 673709992831262724
Valid for: 673708611235921920
Valid for: 673707060090052608
Valid for: 673705679337693185
Valid for: 673700254269775872
Valid for: 673697980713705472
Valid for: 673689733134946305
Valid for:

Valid for: 668932921458302977
Valid for: 668902994700836864
Valid for: 668892474547511297
Valid for: 668872652652679168
Valid for: 668852170888998912
Valid for: 668826086256599040
Valid for: 668815180734689280
Valid for: 668779399630725120
Valid for: 668655139528511488
Valid for: 668645506898350081
Valid for: 668643542311546881
Valid for: 668641109086707712
Valid for: 668636665813057536
Valid for: 668633411083464705
Valid for: 668631377374486528
Valid for: 668627278264475648
Valid for: 668625577880875008
Valid for: 668623201287675904
Valid for: 668620235289837568
Valid for: 668614819948453888
Valid for: 668587383441514497
Valid for: 668567822092664832
Valid for: 668544745690562560
Valid for: 668542336805281792
Valid for: 668537837512433665
Valid for: 668528771708952576
Valid for: 668507509523615744
Valid for: 668496999348633600
Valid for: 668484198282485761
Valid for: 668480044826800133
Valid for: 668466899341221888
Valid for: 668297328638447616
Valid for: 668291999406125056
Valid for:

In [20]:
with open('tweet_json.txt', 'w') as outfile:  
    json.dump(tweet_data, outfile, 
              sort_keys = True,
              indent=4,
              ensure_ascii = False)

In [21]:
temp_tweet = pd.read_json('tweet_json.txt',orient='index')

In [22]:
tweets = temp_tweet[['id','retweet_count','favorite_count']].copy()
tweets.rename(columns={'id': 'tweet_id'}, inplace=True)

## Access data


In [23]:
WeRateDogs

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [24]:
WeRateDogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [25]:
WeRateDogs.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [26]:
TweetImage

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [27]:
TweetImage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [28]:
TweetImage.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [29]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2345 entries, 1991-02-08 13:48:08.022790149 to 1998-04-12 22:37:23.555336193
Data columns (total 3 columns):
tweet_id          2345 non-null int64
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(3)
memory usage: 73.3 KB


In [30]:
tweets.describe()

,tweet_id,retweet_count,favorite_count
count,2.345000e+03,2345.000000,2345.000000
mean,7.422940e+17,3043.018763,8098.874627
std,6.833642e+16,5059.235700,12182.487685
min,6.660209e+17,0.000000,0.000000
25%,6.783802e+17,609.000000,1408.000000
50%,7.189392e+17,1424.000000,3550.000000
75%,7.986979e+17,3548.000000,10019.000000
max,8.924206e+17,77736.000000,143980.000000


In [31]:
WeRateDogs.name.value_counts()

None          745
a              55
Charlie        12
Lucy           11
Oliver         11
Cooper         11
Tucker         10
Penny          10
Lola           10
Bo              9
Winston         9
the             8
Sadie           8
Toby            7
Daisy           7
an              7
Bailey          7
Buddy           7
Stanley         6
Scout           6
Dave            6
Koda            6
Milo            6
Oscar           6
Bella           6
Leo             6
Jax             6
Rusty           6
Jack            6
Alfie           5
             ... 
Dante           1
Rooney          1
Jersey          1
Christoper      1
Tiger           1
Lucky           1
Snoop           1
Dot             1
Ralphy          1
Maude           1
Arlen           1
Severus         1
Monkey          1
Shiloh          1
Willow          1
Lacy            1
Loomis          1
Birf            1
light           1
Tedrick         1
JD              1
Darrel          1
Adele           1
Ralphie         1
Ivar      

In [32]:
WeRateDogs[WeRateDogs.expanded_urls.isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
185,856330835276025856,NaN,NaN,2017-04-24 02:15:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Jenna_Marbles: @dog_rates Thanks for ratin...,8.563302e+17,66699013.0,2017-04-24 02:13:14 +0000,NaN,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None


# Quality

##### `WeRateDogs` table
- `name` is sometimes not an actual name
- wrong data types (`in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `timestamp` and `retweeted_status_timestamp`)
- missing some `expanded_urls`
- 343rd entry is not a dog rating
- some entries should be classified as puppers (missing data)
- some entries are retweets
- Extra characters after '&'

##### `TweetImage` table
- `p1`, `p2`, `p3` inconsistent capitalization (sometimes first letter is capital)
- missing data (only has 2075 entries instead of 2356)

##### `Tweets` table
- missing data (only has 2345 entries instead of 2356)


### Tidiness

- Three data frames `WeRateDogs`, `TweetImage`, and `tweets` should be one (combined table) since all tables' entries are each describing one tweet

##### `WeRateDogs` table
- one variable in four columns (`doggo`, `floofer`, `pupper`, and `puppo`)
- We may want to add a gender column from the text columns in archives dataset
- Get rid of image prediction columns
- Delete unneccessary columns and rename few


## Copy datasets

In [33]:
WeRateDogs_clean = WeRateDogs.copy()
TweetImage_clean = TweetImage.copy()
tweets_clean = tweets.copy()

#### `WeRateDogs`:  
##### Name is sometimes not an actual name


#### Define:

Few dogs names were mentioned as 'a' in the dataset. As we can't assume the name, I just want to replace them with 'None'.
(Note: There are lot of other names which doesnt make any sense like 'an', but I don't to make any changes to them)

#### Code:

In [34]:
WeRateDogs_clean = WeRateDogs_clean.replace('a','None')

#### Test:

In [35]:
# Check if there is no 'a' names
'a' in WeRateDogs_clean['name']

False

#### `WeRateDogs`: 
##### Wrong data types (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, timestamp and retweeted_status_timestamp)


#### Define:

The above mentioned columns are in wrong data types. in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id and retweeted_status_user_id are should be integers and timestamp and retweeted_status_timestamp are should be converted to datetimes.

#### Code:

In [36]:
#Convert float ti int64
WeRateDogs_clean['in_reply_to_status_id'] = WeRateDogs_clean['in_reply_to_status_id'].astype('str')
WeRateDogs_clean['in_reply_to_user_id'] = WeRateDogs_clean['in_reply_to_user_id'].astype('str')
WeRateDogs_clean['retweeted_status_id'] = WeRateDogs_clean['retweeted_status_id'].astype('str')
WeRateDogs_clean['retweeted_status_user_id'] = WeRateDogs_clean['retweeted_status_user_id'].astype('str')

#Convert object to datetime64[ns]
WeRateDogs_clean['timestamp'] = pd.to_datetime(WeRateDogs_clean['timestamp'])
WeRateDogs_clean['retweeted_status_timestamp'] = pd.to_datetime(WeRateDogs_clean['retweeted_status_timestamp'])


#### Test:

In [37]:
WeRateDogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null object
in_reply_to_user_id           2356 non-null object
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null object
retweeted_status_user_id      2356 non-null object
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: dateti

#### `WeRateDogs`: 
##### Missing some expanded_urls

#### Define:

There are 60 url's missing in the dataset. We can create the url's with the help of tweet id.

#### Code:

In [38]:
def clean_url(dogs):
    if pd.notnull(dogs['expanded_urls']):
        return dogs['expanded_urls']
    else:
        url = 'https://twitter.com/dog_rates/status/' + str(dogs.tweet_id) + '/photo/1'
        return url
    
WeRateDogs_clean['expanded_urls'] = WeRateDogs_clean.apply(clean_url, axis=1)

#### Test

In [39]:
WeRateDogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null object
in_reply_to_user_id           2356 non-null object
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null object
retweeted_status_user_id      2356 non-null object
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2356 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: dateti

#### `WeRateDogs`: 
##### 343rd entry is not a dog rating

#### Define:

Remove 343th entry from dataframe

#### Code:

In [40]:
WeRateDogs_clean = WeRateDogs_clean.drop(WeRateDogs_clean.index[342])

#### Test

In [41]:
WeRateDogs_clean.iloc[340:343]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
340,832215909146226688,nan,nan,2017-02-16 13:11:49,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: This is Logan, the Chow who liv...",7.8670908285e+17,4196983835.0,2016-10-13 23:23:56,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
341,832215726631055365,nan,nan,2017-02-16 13:11:05,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Moreton. He's the Good ...,7.93286476302e+17,4196983835.0,2016-11-01 03:00:09,https://twitter.com/dog_rates/status/793286476...,13,10,Moreton,None,None,None,None
343,832040443403784192,nan,nan,2017-02-16 01:34:34,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Klein. These pics were ...,7.69940425801e+17,4196983835.0,2016-08-28 16:51:16,https://twitter.com/dog_rates/status/769940425...,12,10,Klein,None,None,None,None


#### `WeRateDogs`: 
##### Some entries should be classified as puppers (missing data)

#### Define:

The pupper column can be filled based on the puppers text idenfied in the 'text' column 

#### Code:

In [42]:
def clean_pupper(dogs):
    if 'puppers' in dogs['text']:
        return 'pupper'
    else:
        return dogs['pupper']
    
WeRateDogs_clean['pupper'] = WeRateDogs_clean.apply(clean_pupper, axis=1)

#### Test

In [43]:
WeRateDogs_clean.pupper.value_counts()

None      2075
pupper     280
Name: pupper, dtype: int64

#### `WeRateDogs`: 
##### Some entries are retweets

#### Define:

Find the retweeted tweets from the tet column and delete them. Identify the retweets using '^RT @'

#### Code:

In [44]:
WeRateDogs_clean = WeRateDogs_clean[WeRateDogs_clean.text.str.contains("^RT @") == False]

#### Test

In [45]:
WeRateDogs_clean[WeRateDogs_clean['text'].str.contains('^RT @')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


#### `WeRateDogs`: 
##### Some ratings are decimal

#### Define:

Get the decimal ratings by matching the text pattern "#.#/#" and save as new rating (only denominators had decimals)

#### Code:

In [46]:
def clean_rating(dogs):
    search_pattern = '(\d+\.\d+)\/(\d+)' 
    rating = re.search(search_pattern, dogs['text'])
    if rating != None:
        dogs['rating_numerator'] =  rating.group(1)
        dogs['rating_denominator'] = rating.group(2)
        
    return dogs



WeRateDogs_clean = WeRateDogs_clean.apply(clean_rating, axis=1)

#### Test

In [47]:
WeRateDogs_clean[WeRateDogs_clean.text.str.contains('(\d+\.\d+)\/(\d+)')].rating_numerator

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


45       13.5
695      9.75
763     11.27
1689      9.5
1712    11.26
Name: rating_numerator, dtype: object

#### `WeRateDogs`: 
##### Extra characters after '&'

#### Define:

Delete the extra characters after '&'.

#### Code:

In [48]:
WeRateDogs_clean['text'] = WeRateDogs_clean.text.str.replace('&amp','&')

#### Test

In [49]:
WeRateDogs_clean[WeRateDogs_clean.text.str.contains('&amp')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


#### `TweetImage`: 
##### p1, p2, p3 inconsistent capitalization (sometimes first letter is capital)

#### Define:

Capitalize the first letter of the each word

#### Code:

In [50]:
TweetImage['p1'] = TweetImage['p1'].str.lower()
TweetImage['p2'] = TweetImage['p2'].str.lower()
TweetImage['p3'] = TweetImage['p3'].str.lower()

#### Test

In [51]:
TweetImage

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,welsh_springer_spaniel,0.465074,True,collie,0.156665,True,shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,german_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,rottweiler,0.243682,True,doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,bernese_mountain_dog,0.651137,True,english_springer,0.263788,True,greater_swiss_mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


#### `TweetImage`: 
##### Missing data (only has 2075 entries instead of 2356)

#### Define:

Create new dataframe by merging data from image predictions into a copy of twitter archive if the image prediction's tweet is already found in the twitter archive

#### Code:

In [52]:
WeRateDogs_master = pd.merge(WeRateDogs_clean, TweetImage_clean, on='tweet_id', how='left')

#### Test

In [53]:
# Copy the tweet id into a list
tweet_id = TweetImage_clean.tweet_id


for each in tweet_id:
    # get the tweet ids which are not avaiable in WeRateDogs_clean dataframe
    if each not in WeRateDogs_clean.tweet_id:
        # check if the tweet id is in new dataframe WeRateDogs_master
        if each in WeRateDogs_master.tweet_id:
            print('Paradox! ID#{} in final DF but shouldn\'t be'.format(each))
            
            
WeRateDogs_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2173
Data columns (total 28 columns):
tweet_id                      2174 non-null int64
in_reply_to_status_id         2174 non-null object
in_reply_to_user_id           2174 non-null object
timestamp                     2174 non-null datetime64[ns]
source                        2174 non-null object
text                          2174 non-null object
retweeted_status_id           2174 non-null object
retweeted_status_user_id      2174 non-null object
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 2174 non-null object
rating_numerator              2174 non-null object
rating_denominator            2174 non-null object
name                          2174 non-null object
doggo                         2174 non-null object
floofer                       2174 non-null object
pupper                        2174 non-null object
puppo                         2174 non-null object
jpg_url       

#### `Tweets`: 
##### Missing data (only has 2345 entries instead of 2356)

#### Define:

Merge data from tweet info into a final dataframe if the tweet info's tweet is already found in the final dataframe (twitter archive)

Note that the 9 missing tweets are because the tweets have been removed.

#### Code:

In [54]:
WeRateDogs_master = pd.merge(WeRateDogs_master, tweets, on='tweet_id', how='left')

#### Test

In [55]:
# Copy the tweet id into a list
tweet_id = tweets.tweet_id


for each in tweet_id:
    # get the tweet ids which are not avaiable in WeRateDogs_clean dataframe
    if each not in WeRateDogs_clean.tweet_id:
        # check if the tweet id is in new dataframe WeRateDogs_master
        if each in WeRateDogs_master.tweet_id:
            print('Paradox! ID#{} in final DF but shouldn\'t be'.format(each))
            

WeRateDogs_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2173
Data columns (total 30 columns):
tweet_id                      2174 non-null int64
in_reply_to_status_id         2174 non-null object
in_reply_to_user_id           2174 non-null object
timestamp                     2174 non-null datetime64[ns]
source                        2174 non-null object
text                          2174 non-null object
retweeted_status_id           2174 non-null object
retweeted_status_user_id      2174 non-null object
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 2174 non-null object
rating_numerator              2174 non-null object
rating_denominator            2174 non-null object
name                          2174 non-null object
doggo                         2174 non-null object
floofer                       2174 non-null object
pupper                        2174 non-null object
puppo                         2174 non-null object
jpg_url       

### Copy the WeRateDogs_master

In [56]:
WeRateDogs_master_copy = WeRateDogs_master.copy()

#### `WeRateDogs_master`: 
##### One variable in four columns (doggo, floofer, pupper, and puppo)

#### Define:

Concatenate doggo, floofer, pupper, and puppo columns data to one column and delete all of them.

#### Code:

In [57]:
WeRateDogs_master_copy['dog_type'] = WeRateDogs_master_copy['doggo'].fillna('') + ',' + WeRateDogs_master_copy['floofer'].fillna('') + ',' + WeRateDogs_master_copy['pupper'].fillna('') + ',' + WeRateDogs_master_copy['puppo'].fillna('')
WeRateDogs_master_copy['dog_type'] = WeRateDogs_master_copy['dog_type'].str.replace('None,','')
WeRateDogs_master_copy['dog_type'] = WeRateDogs_master_copy['dog_type'].str.replace(',None','')
WeRateDogs_master_copy['dog_type'] = WeRateDogs_master_copy['dog_type'].str.replace('None','')

WeRateDogs_master_copy = WeRateDogs_master_copy.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

#### Test

In [58]:
WeRateDogs_master_copy.dog_type.value_counts()

                 1807
pupper            247
doggo              75
puppo              24
doggo,pupper       10
floofer             9
doggo,floofer       1
doggo,puppo         1
Name: dog_type, dtype: int64

#### `WeRateDogs_master`: 
##### We may want to add a gender column from the text columns in archives dataset

#### Define:

Find out the gender related text in the 'text' column and add Male or Female in a new column

#### Code:

In [59]:
male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in WeRateDogs_master_copy['text']:
    # male
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    else:
        dog_gender.append('NaN')


WeRateDogs_master_copy['dog_gender'] = dog_gender

#### Test

In [60]:
WeRateDogs_master_copy.dog_gender.value_counts()

NaN       1273
male       668
female     233
Name: dog_gender, dtype: int64

In [61]:
WeRateDogs_master_copy.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_type,dog_gender
0,892420643555336193,nan,nan,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,nan,nan,NaT,https://twitter.com/dog_rates/status/892420643...,...,bagel,0.085851,False,banana,0.076110,False,NaN,NaN,,NaN
1,892177421306343426,nan,nan,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,nan,nan,NaT,https://twitter.com/dog_rates/status/892177421...,...,Pekinese,0.090647,True,papillon,0.068957,True,NaN,NaN,,female
2,891815181378084864,nan,nan,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,nan,nan,NaT,https://twitter.com/dog_rates/status/891815181...,...,malamute,0.078253,True,kelpie,0.031379,True,4213.0,25143.0,,male
3,891689557279858688,nan,nan,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,nan,nan,NaT,https://twitter.com/dog_rates/status/891689557...,...,Labrador_retriever,0.168086,True,spatula,0.040836,False,8762.0,42317.0,,female
4,891327558926688256,nan,nan,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,nan,nan,NaT,https://twitter.com/dog_rates/status/891327558...,...,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,9524.0,40473.0,,male


#### `WeRateDogs_master`: 
##### Get rid of image prediction columns

#### Define:

Find out be best predicted and confident dog type from p1,p2,p3 and p4 columns and create new columns. THen after delete the old one's.

#### Code:

In [62]:
predictions = []
confidence = []

def get_pred_conf(row):
    if row.p1_dog == True:
        predictions.append(row['p1'])
        confidence.append(row['p1_conf'])
    elif row.p2_dog == True:
        predictions.append(row['p2'])
        confidence.append(row['p2_conf'])
    elif row.p3_dog == True:
        predictions.append(row['p3'])
        confidence.append(row['p3_conf'])
    else:
        predictions.append('NaN')
        confidence.append(0)


WeRateDogs_master_copy.apply(get_pred_conf, axis=1)
WeRateDogs_master_copy['predictions'] = predictions
WeRateDogs_master_copy['confidence'] = confidence

In [63]:
WeRateDogs_master_copy = WeRateDogs_master_copy.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

#### Test

In [64]:
WeRateDogs_master_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2173
Data columns (total 20 columns):
tweet_id                      2174 non-null int64
in_reply_to_status_id         2174 non-null object
in_reply_to_user_id           2174 non-null object
timestamp                     2174 non-null datetime64[ns]
source                        2174 non-null object
text                          2174 non-null object
retweeted_status_id           2174 non-null object
retweeted_status_user_id      2174 non-null object
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 2174 non-null object
rating_numerator              2174 non-null object
rating_denominator            2174 non-null object
name                          2174 non-null object
jpg_url                       1994 non-null object
retweet_count                 1420 non-null float64
favorite_count                1420 non-null float64
dog_type                      2174 non-null object
dog_gender  

#### `WeRateDogs_master`: 
##### Delete unneccessary columns and rename few

#### Define:

Delete unneccessary columns in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp.

Also rename retweet_count to tweet_retweet, favorite_count to tweet_favorite, timestamp to tweet_time, source to tweet_source, text to tweet_text, expanded_urls to tweet_url, name to dog_name

#### Code:

In [65]:
WeRateDogs_master_copy = WeRateDogs_master_copy.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], 1)

WeRateDogs_master_copy.rename(columns={'retweet_count':'tweet_retweet', 'favorite_count':'tweet_favorite', 'timestamp':'tweet_time', 'source':'tweet_source', 'text':'tweet_text', 'expanded_urls':'tweet_url', 'name':'dog_name'}, inplace=True)

#### Test

In [66]:
WeRateDogs_master_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2173
Data columns (total 15 columns):
tweet_id              2174 non-null int64
tweet_time            2174 non-null datetime64[ns]
tweet_source          2174 non-null object
tweet_text            2174 non-null object
tweet_url             2174 non-null object
rating_numerator      2174 non-null object
rating_denominator    2174 non-null object
dog_name              2174 non-null object
jpg_url               1994 non-null object
tweet_retweet         1420 non-null float64
tweet_favorite        1420 non-null float64
dog_type              2174 non-null object
dog_gender            2174 non-null object
predictions           2174 non-null object
confidence            2174 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(10)
memory usage: 271.8+ KB


### Storing the master data

In [67]:
WeRateDogs_master_copy.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')

In [68]:
df_master = pd.read_csv('twitter_archive_master.csv')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2174 entries, 0 to 2173
Data columns (total 15 columns):
tweet_id              2174 non-null int64
tweet_time            2174 non-null object
tweet_source          2174 non-null object
tweet_text            2174 non-null object
tweet_url             2174 non-null object
rating_numerator      2174 non-null float64
rating_denominator    2174 non-null int64
dog_name              2174 non-null object
jpg_url               1994 non-null object
tweet_retweet         1420 non-null float64
tweet_favorite        1420 non-null float64
dog_type              367 non-null object
dog_gender            901 non-null object
predictions           1686 non-null object
confidence            2174 non-null float64
dtypes: float64(4), int64(2), object(9)
memory usage: 254.8+ KB
